In [1]:
import numpy as np 
import pandas as pd
import os
import csv

def read_data(input_file):
        """Read a BIO data!"""
        #rf = open(input_file,'r')
        #rf = csv.reader(input_file, delimiter="\t", quotechar=None)
        with open(input_file, 'r') as f:
            rf = list(csv.reader(f))
            lines = [];words = [];labels = []; sl=[];ll=[];
            for line in rf[1:]:
                word = line[0]
                label = line[1]
                # here we dont do "DOCSTART" check
                if len(word)<=1 and word[-1] == '.':
                    l = [label for label in labels if len(label) > 0]
                    w = ' '.join([word for word in words if len(word) > 0])
                    lines.append((l,w))
                    words=[]
                    labels = []
                    sl.append(w)
                    ll.append(l)
                words.append(word)
                labels.append(label)
        return sl,ll

In [2]:
#dataset = pd.read_csv("./data/train.csv") #[word, label
sentences, labels = read_data("./data/train.csv")
epochs = 5
max_grad_norm = 1.0
tags_vals = ["[PAD]","PATIENT", "DATE", "HOSPITAL", "PHONE", "DOCTOR", "LOCATION", "AGE", "ID", "NONE", "X","[CLS]","[SEP]"]
tag2idx = {t: i for i, t in enumerate(tags_vals)}
MAX_LEN = 128
bs =32 #batch_size
CONFIG_FILE = ""
CHECKPOINT_FILE = "./mimicpretraining/out/model.ckpt-0"

In [3]:
sentences

['113416550 PRGH 13523357 630190 6/7 1999 12:00:00 AM Discharge Summary Signed DIS Admission Date : 06/07 1999 Report Status : Signed Discharge Date : 06/13 1999 HISTORY OF PRESENT ILLNESS : Essentially , Mr. Cornea is a 60 year old male who noted the onset of dark urine during early January',
 '. He underwent CT and ERCP at the Lisonatemi Faylandsburgnic, Community Hospital with a stent placement and resolution of jaundice',
 '. He underwent an ECHO and endoscopy at Ingree and Ot of Weamanshy Medical Center on April 28',
 '. He was found to have a large , bulging , extrinsic mass in the lesser curvature of his stomach',
 '. Fine needle aspiration showed atypical cells , positively reactive mesothelial cells',
 '. Abdominal CT on April 14 , showed a 12 x 8 x 8 cm mass in the region of the left liver , and appeared to be from the lesser curvature of the stomach or left liver',
 '. He denied any nausea , vomiting , anorexia , or weight loss',
 '. He states that his color in urine or in s

In [5]:
import torch
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertForTokenClassification, BertAdam
from tqdm import tqdm, trange

Using TensorFlow backend.


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

In [7]:
torch.cuda.get_device_name(0) 

'Tesla K80'

In [8]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

100%|██████████| 231508/231508 [00:00<00:00, 34798413.50B/s]


In [9]:
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print(tokenized_texts[0])

['113', '##41', '##65', '##50', 'pr', '##gh', '135', '##23', '##35', '##7', '630', '##19', '##0', '6', '/', '7', '1999', '12', ':', '00', ':', '00', 'am', 'discharge', 'summary', 'signed', 'di', '##s', 'admission', 'date', ':', '06', '/', '07', '1999', 'report', 'status', ':', 'signed', 'discharge', 'date', ':', '06', '/', '13', '1999', 'history', 'of', 'present', 'illness', ':', 'essentially', ',', 'mr', '.', 'corn', '##ea', 'is', 'a', '60', 'year', 'old', 'male', 'who', 'noted', 'the', 'onset', 'of', 'dark', 'urine', 'during', 'early', 'january']


In [10]:
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

W0822 08:38:46.771541 139677834598144 tokenization.py:126] Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (817 > 512). Running this sequence through BERT will result in indexing errors
W0822 08:38:46.773687 139677834598144 tokenization.py:126] Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (854 > 512). Running this sequence through BERT will result in indexing errors
W0822 08:38:46.782233 139677834598144 tokenization.py:126] Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (1209 > 512). Running this sequence through BERT will result in indexing errors
W0822 08:38:46.784674 139677834598144 tokenization.py:126] Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (539 > 512). Running this sequence through BERT will result in indexing errors
W0822 08:38:46.785583 139677834

In [12]:
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
                     maxlen=MAX_LEN, value=tag2idx["X"], padding="post",
                     dtype="long", truncating="post")

In [13]:
attention_masks = [[float(i>0) for i in ii] for ii in input_ids]

In [14]:
tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids, tags, 
                                                            random_state=2018, test_size=0.1)
tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.1)

In [15]:
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)

In [16]:
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)

In [17]:
config = BertConfig.from_json_file(CONFIG_FILE)
model = BertForPreTraining(config)
model = load_tf_weights_in_bert(model, CHECKPOINT_FILE)
#model = BertForTokenClassification.from_pretrained("bert-base-uncased", num_labels=len(tag2idx))

100%|██████████| 407873900/407873900 [00:07<00:00, 56350147.02B/s]


In [18]:
model.cuda()

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
      

In [19]:
FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters()) 
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = Adam(optimizer_grouped_parameters, lr=3e-5)

In [21]:
from seqeval.metrics import f1_score

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=2).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
for _ in trange(epochs, desc="Epoch"):
    # TRAIN loop
    model.train()
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        # forward pass
        loss = model(b_input_ids, token_type_ids=None,
                     attention_mask=b_input_mask, labels=b_labels)
        # backward pass
        loss.backward()
        # track train loss
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        # update parameters
        optimizer.step()
        model.zero_grad()
    # print train loss per epoch
    print("Train loss: {}".format(tr_loss/nb_tr_steps))
    # VALIDATION on validation set
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions , true_labels = [], []
    for batch in valid_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        with torch.no_grad():
            tmp_eval_loss = model(b_input_ids, token_type_ids=None,
                                  attention_mask=b_input_mask, labels=b_labels)
            logits = model(b_input_ids, token_type_ids=None,
                           attention_mask=b_input_mask)
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
        true_labels.append(label_ids)
        
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        eval_loss += tmp_eval_loss.mean().item()
        eval_accuracy += tmp_eval_accuracy
        
        nb_eval_examples += b_input_ids.size(0)
        nb_eval_steps += 1
    eval_loss = eval_loss/nb_eval_steps
    print("Validation loss: {}".format(eval_loss))
    print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
    pred_tags = [tags_vals[p_i] for p in predictions for p_i in p]
    valid_tags = [tags_vals[l_ii] for l in true_labels for l_i in l for l_ii in l_i]
    print("F1-Score: {}".format(f1_score(pred_tags, valid_tags)))

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
model.eval()
predictions = []
true_labels = []
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
for batch in valid_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    with torch.no_grad():
        tmp_eval_loss = model(b_input_ids, token_type_ids=None,
                              attention_mask=b_input_mask, labels=b_labels)
        logits = model(b_input_ids, token_type_ids=None,
                       attention_mask=b_input_mask)
        
    logits = logits.detach().cpu().numpy()
    predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
    label_ids = b_labels.to('cpu').numpy()
    true_labels.append(label_ids)
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)

    eval_loss += tmp_eval_loss.mean().item()
    eval_accuracy += tmp_eval_accuracy

    nb_eval_examples += b_input_ids.size(0)
    nb_eval_steps += 1

pred_tags = [[tags_vals[p_i] for p_i in p] for p in predictions]
valid_tags = [[tags_vals[l_ii] for l_ii in l_i] for l in true_labels for l_i in l ]
print("Validation loss: {}".format(eval_loss/nb_eval_steps))
print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
print("Validation F1-Score: {}".format(f1_score(pred_tags, valid_tags)))